In [1]:
import requests
import pandas as pd
import numpy as np
import sys, os, re, time
import json, csv
from bs4 import BeautifulSoup
from collections import defaultdict

In [2]:
API_SEARCH_URL = 'http://api.nytimes.com/svc/movies/v2/reviews/search.json?'
API_KEY = 'REPLACE_KEY_HERE'
OFFSET = 27248

In [3]:
def getReviews(params={}):
    query_params = []
    
    for k in params:
        query_params.append(k + '=' + str(params[k]))
    
    if 'api_key' not in params:
        query_params.append('api-key' + '=' + API_KEY)
    
    result = requests.get(API_SEARCH_URL + '&'.join(query_params)).json()
    
    time.sleep(6)
    
    return result

In [4]:
with open('nyt_reviews.csv', 'w+') as fo:
    fieldnames = ['display_title', 'headline', 'byline', 'link', 'mpaa_rating',
                  'opening_date', 'publication_date', 'date_updated',
                  'critics_pick', 'summary_short']
    
    writer = csv.writer(fo)
    writer.writerow(fieldnames)

    params = defaultdict(str)

    offset = OFFSET
    while True:
        params['offset'] = offset

#         print(offset)
        
        response = getReviews(params)
        
        if ('num_results' not in response):
            if ('hasMore' not in response):
                print('ERROR:')
                print('offset=' + str(offset))
                print(response)
                break
            else:
                hasMore = response['hasMore']
                num_results = len(response['results'])
        else:
            num_results = response['num_results']
            if ('hasMore' not in response):
                hasMore = True
            else:
                hasMore = response['hasMore']

        if (num_results == 0) or (hasMore is not True):
            break
            
#         print(num_results)

        for result in response['results']:
            line = [result['display_title'], result['headline'], result['byline'],
                    result['link']['url'], result['mpaa_rating'], result['opening_date'],
                    result['publication_date'], result['date_updated'],
                    result['critics_pick'], result['summary_short']]
            writer.writerow(line)

#         print()

        offset += num_results